In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

In [75]:
class DNN(nn.Module):
    def __init__(self, input_dim=1, output_dim=6, hidden_layers=[64,64]):
        super(DNN, self).__init__()
        net = nn.ModuleList([])

        net.append(nn.Linear(input_dim, hidden_layers[0]))
        net.append(nn.Tanh())

        for i in range(len(hidden_layers) - 1):
            net.append(nn.Linear(hidden_layers[i], hidden_layers[i+1]))
            net.append(nn.Tanh())

        net.append(nn.Linear(hidden_layers[-1], output_dim))

        self.net = nn.Sequential(*net)

    def forward(self, x):
        return self.net(x)
    
class Parameter(nn.Module):
    def __init__(self, parameter_dim=2):
        super(Parameter, self).__init__()
        self.parameter = nn.ParameterList([nn.Parameter(torch.randn(parameter_dim))])

    def forward(self, x):
        return self.parameter[0]

In [3]:
data = np.loadtxt('data_lorentz/train.txt')

In [4]:
print(data.shape)
print(np.max(data))
print(np.min(data))

(1500, 4)
1.6881171100004457
-1.6882507966870368


In [16]:
targets = torch.tensor(data)

for i in range(targets.shape[1]):
    targets[:, i] = 2 * (targets[:, i] - torch.min(targets[:, i])) / (torch.max(targets[:, i]) - torch.min(targets[:, i])) - 1

inputs = torch.linspace(-1, 1, targets.shape[0])
inputs = inputs.unsqueeze(1)

In [76]:
model = DNN(input_dim=1, output_dim=4, hidden_layers=[64, 64, 64])
para_estimates = Parameter(parameter_dim=2)

In [65]:
batch_size = 128
dataset = torch.utils.data.TensorDataset(inputs, targets)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
def diff_eq_loss(u_t, u, m, q):
    v = u[:, :2]
    x = u[:, 2:]
    rhs = torch.cat((m * v, q * x / (100 * (x[:, 0:1]**2 + x[:, 1:2]**2)**1.5)), dim=1)

    matrix = torch.zeros(4, 4)
    matrix[0, 2] = -1 / m
    matrix[1, 3] = -1 / m
    matrix[2, 0] = 1 / m
    matrix[3, 1] = 1 / m
    matrix[0, 1] = q * torch.sqrt(x[:, 0]**2 + x[:, 1]**2) / m**2
    matrix[1, 0] = -q * torch.sqrt(x[:, 0]**2 + x[:, 1]**2) / m**2


    loss = torch.mean(torch.abs(u_t.T - torch.matmul(rhs.T, matrix.T)))

    return loss


In [90]:
from itertools import chain
optimizer = torch.optim.AdamW(chain(model.parameters(), para_estimates.parameters()), lr=1e-4)

epochs = 100
factor_physics = 0.1

for i in range(epochs):
    for batch in dataloader:
        t, u = batch
        t.requires_grad_()
        output = model(t)
        parameters = para_estimates(torch.randn(1))
        m = parameters[0]
        q = parameters[1]

        u_t = torch.autograd.grad(output, t, grad_outputs=torch.ones_like(output), create_graph=True)[0]

        loss_data = torch.mean((output[:, :4] - u)**2)
        loss_equation = diff_eq_loss(u_t[:, :4], u, m, q)

        loss = loss_data + factor_physics * loss_equation
        loss.backward()
        optimizer.step()

        print(f'Epoch: {i} | Data Loss: {loss_data} | Physics Loss: {loss_equation}')

RuntimeError: The size of tensor a (2) must match the size of tensor b (128) at non-singleton dimension 1

In [79]:
print(m.shape)

torch.Size([])


In [84]:
print(q)

tensor(0.5205, grad_fn=<SelectBackward0>)


In [44]:
a = torch.randn((128, 2))
b = torch.randn((128, 2))
print(torch.cat((a, b), dim=1))

tensor([[ 1.4298e+00,  1.6585e-01,  1.2683e-01,  4.4790e-01],
        [ 1.5370e+00,  1.3696e+00,  2.6426e+00, -5.6680e-01],
        [-1.1812e+00, -1.6765e+00,  2.7335e-01,  1.6729e-01],
        [ 1.3919e+00, -1.3108e+00, -9.7579e-01, -1.8688e+00],
        [ 2.2220e-02,  1.0162e+00, -8.5708e-01, -1.4967e+00],
        [-7.4280e-01,  2.3078e-01,  2.4906e+00,  8.9956e-01],
        [-1.0917e+00,  3.7615e-01, -2.7556e-02, -1.1070e+00],
        [ 4.7760e-01, -3.5378e-01, -1.2565e+00,  9.4480e-01],
        [-7.1277e-01, -6.0166e-01,  1.1473e+00, -1.8579e+00],
        [-8.3687e-01, -4.7155e-01,  4.0082e-01, -3.9907e-01],
        [ 1.9920e-01,  1.1962e+00, -2.7494e-01, -5.9108e-01],
        [ 2.0649e-01,  1.2994e+00, -5.7723e-02, -1.7688e-01],
        [ 1.3391e-01,  9.3440e-01,  1.6281e-03, -1.3757e+00],
        [ 8.7636e-01, -8.8103e-01,  8.4610e-01, -2.5312e-01],
        [-1.9729e-03,  1.8605e-01, -7.3161e-01,  9.9392e-02],
        [-5.4423e-01,  8.3947e-01,  4.7877e-01, -5.1627e-01],
        

In [32]:
print(u_t[1])

IndexError: tuple index out of range

In [27]:
print(u_t.shape)

AttributeError: 'NoneType' object has no attribute 'shape'

In [22]:
print(output.shape)

torch.Size([128, 6])
